# Generation of synthetic promoters for *C. cinerea* from upstream sequences
## Purpose
This notebook extracts upstream sequences from the *C. cinerea* genome and trains a neural generator. It then generates synthetic promoters based on this generator.
## Extraction of upstream sequences

In [1]:
#load required libraries
import pandas as pd
from Bio import SeqIO, AlignIO, BiopythonParserWarning
import shutil
import subprocess
import os
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
import sys
import warnings


os.chdir("..") #change directory to root for relative improrts

In [3]:
#import from utils module
from src.utils import extract_upstream_sequences
warnings.filterwarnings("ignore", category=BiopythonParserWarning)

upstream_seq_df = extract_upstream_sequences("genome/amutbmut.gbk", 1000, ["CDS"])
print(upstream_seq_df.head())

            id                                           sequence
0  CC2G_000001  CTCCCAAAAAGCGTAAGTCCTATTCTCTTTCTACTACTATCTTTTG...
1  CC2G_000002  CTTGGCCTTAGTGACAACACCTTGTTCGCCACTGCTCTATCTAACT...
2  CC2G_000003  CAAGCTCTACCGGCGAAGTGATTTGCCAATTCTTCTGTTGCCGGCG...
3  CC2G_000004  TTCAAGTGTTTTATCACAGTATCTAACATCATGACTTCACCGATGG...
4  CC2G_000005  AGACTAGCAAAGACTATAAAACAAACGGAATACATGCGCATGATAC...
